<a href="https://colab.research.google.com/github/raphaele42/Python-examples/blob/master/KNNimplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of the k nearest neighbours algorithm

k nearest neighbours algorithm applied on a sample of ~4000 observations the TunedIT data set, classified as either rock or not rock. There are 191 characteristics for each observations. The last variable RockOrNot denotes if the observation is either rock (1) or not rock (0).

In [0]:
# Import packages
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import numpy.random as npr
import random
from scipy.spatial.distance import pdist, squareform #for euclidean distance

# Data Preparation

Load in the data and look at the data. Note: tunedit_genres.csv is available in the current directory.

In [2]:
data = pd.read_csv('tunedit_genres.csv')
data.head()  #view 5 first observations
data.describe()  #summary stats
data.dtypes  #check variables data type

PAR_TC                    float64
PAR_SC                    float64
PAR_SC_V                  float64
PAR_ASE1                  float64
PAR_ASE2                  float64
                           ...   
PAR_2RMS_TCD_10FR_MEAN    float64
PAR_2RMS_TCD_10FR_VAR     float64
PAR_3RMS_TCD_10FR_MEAN    float64
PAR_3RMS_TCD_10FR_VAR     float64
RockOrNot                   int64
Length: 192, dtype: object

Determine percentage of the songs that are rock songs (to 1 d.p.).

In [3]:
# Frequency table for the RockOrNot variable
table = data.RockOrNot.value_counts()
# Computing the percentage of rock songs
freqRock = round(table[1]/len(data)*100, 1)
print("The percentage of rock songs is: ",freqRock)
# Ans: 48.8%

The percentage of rock songs is:  48.8


'RockOrNot' is the classification variable. We need to separate it from the other variables. We will separate the data into a DataFrames X and a Series y, where:
 
*   X contains a standardised version of all variables except for the classification variable ('RockOrNot')
*   y contains only the classification variable



In [0]:
# Series for classification variable
y = pd.Series(data['RockOrNot'].values)
# Dataframe for predictive variables
X = pd.DataFrame(data.drop('RockOrNot', axis=1).values)

We also need to standardise the variables in X, by subtracting the mean and dividing by the standard deviation.

In [6]:
# Standardise all values in X
X = (X - X.mean()) / X.std()
# Check the standardisation was correct
X.mean() #should be 0
X.std()  #should be 1

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
186    1.0
187    1.0
188    1.0
189    1.0
190    1.0
Length: 191, dtype: float64

# Splitting the data into training and data sets.

The model will be fitted on the training set (75% of the original set). The remaining 25% is the test set, used to determine the performance of the model. 

X and y must be divided randomly into training and test sets. The split must match, for all the observations from X included in the training set, the corresponding observation in y must be included in the training set.

In [0]:
# set a seed for the random split can be replicated
random.seed(123)
fullInd = list(X.index)   #full list of indices
n = len(fullInd)
trainProp = round(n*0.75)    #set number of items for training set
#Select random items for training indices
trainInd = random.sample(fullInd, trainProp)
#Define test set indices as complement of training indices
testInd = [x for i, x in enumerate(fullInd) if i not in trainInd]
#Build training and test dfs based on indices lists
trainX = X.iloc[trainInd]
trainy = y[trainInd]
testX = X.iloc[testInd]
testy = y[testInd]
# Test the split is correct by checking length of training and test sets
# Expected split: 2999 / 1000
len(testX)
len(testy)
len(trainX)
len(trainy)

# KNN function

The first function runs kNN (k Nearest Neighbours) on the data sets. How the algorithm works :
1.

1.   A value of k is chosed (usually odd)
2.   For each observation, find its k closest neighbours
3.   Take the majority vote (mean) of these neighbours
4.   Classify the observation based on majority vote

The distance between observations is based on the standard Euclidean distance.

The function outputs a series y_star of predicted classification values.


In [0]:
def kNN(X,y,k):
    # Find the number of obsvervation
    n = len(y)
    # Set up return values
    y_star = pd.Series(0, index=np.arange(n))
    # Calculate the distance matrix for the observations in X
    dist = pdist(X, metric='euclidean')
    dist = squareform(dist)    
    # Make all the diagonals very large so an observation can't choose itself as a closest neighbour
    dist[dist == 0] = 300
    # Loop through each observation to create predictions
    kMean=0
    for i in range(0, n): 
        # Find the y values of the k nearest neighbours
        y_nearest = 0
        allNeighbours = pd.Series(dist[i, ])
        for j in range(1, k):
            closestInd = allNeighbours.idxmin()
            newClosest = y.iloc[closestInd]
            y_nearest = y_nearest + newClosest
            allNeighbours = allNeighbours.drop([closestInd])
            #compute average of neighbours class
            #round to nearest integer so result is 0 or 1
            kMean = int(round(y_nearest / k))
        # Allocate to y_star
        y_star.iloc[i] = kMean
    return y_star


# Select the best k

The best value of k is the one for which the misclassification rate is the smallest. This function kNN_select runs a kNN classification for a range of k values, and compute the misclassification rate for each.

It returns a Series mis_class_rates, indexed by k, with the misclassification rates for each k value in k_vals.

In [0]:
def kNN_select(X,y,k_vals):
    m = len(k_vals)
    #Initialise the series to store the misclassification rates
    mis_class_rates = pd.Series(0, index=np.arange(m))
    #Loop through all k values to apply KNN for each
    for i in range(0, m): 
        kValue = k_vals[i]
        predRes = kNN(X,y,kValue)
        #Determine misclassification rate for this value of k
        misClassRateNew = round(len(y[y.values != predRes.values]) / len(y), 2)
        mis_class_rates.loc[i] = misClassRateNew
    mis_class_rates.index = k_vals
    return mis_class_rates

# Generalised Function

This general function does:
1.   Separate the data sets X and y into training and test sets where the number in each is specified by 'percent_train'.
2.   Run the k nearest neighbours classification on the training data, for a set of k values, computing the mis-classification rate for each k. 
3.   Find the k that gives the lowest mis-classification rate for the training data, and hence, the classification with the best fit to the data.
4.   Use the best k value to run the k nearest neighbours classification on the test data, and calculate the mis-classification rate for test data.

The function returns the mis-classification rate for a k nearest neighbours classification on the test data, using the best k value for the training data. It uses the function defined above: KNN and KNN_select.

In [0]:
def kNN_classification(df,class_column,seed,percent_train,k_vals):
    # df            - DataFrame to 
    # class_column  - column of df to be used as classification variable, should
    #                 specified as a string  
    # seed          - seed value for creating the training/test sets
    # percent_train - percentage of data to be used as training data
    # k_vals        - set of k values to be tests for best classification
    
    # Separate X and y
    # Series for classification variable
    y = pd.Series(df[class_column].values)
    # Dataframe for predictive variables
    X = pd.DataFrame(df.drop(class_column, axis=1).values)
    # Standardise all values in X
    X = (X - X.mean()) / X.std()
    
    # Divide into training and test
    random.seed(seed)
    fullInd = list(X.index)   #full list of indices
    n = len(fullInd)
    trainProp = round(n*percent_train)    #set number of items for training test
    #Select random items for training indices
    trainInd = random.sample(fullInd, trainProp)
    #Define test set indices as complement of training indices
    testInd = [x for i, x in enumerate(fullInd) if i not in trainInd]
    #Build training and test dfs based on indices lists
    trainX = X.iloc[trainInd]
    trainy = y[trainInd]
    testX = X.iloc[testInd]
    testy = y[testInd]
    
    # Compute the mis-classification rates for each of the values in k_vals
    MisclassRates = kNN_select(trainX,trainy,k_vals) 
    
    # Find the best k value, by finding the minimum entry of mis_class_rates 
    bestK = MisclassRates.idxmin()
    
    # Run the classification on the test set to see how well the 'best fit'
    # classifier does on new (test) data generated from the same source
    predResTest = kNN(testX, testy, bestK)
    
    # Calculate the mis-classification rates for the test data
    mis_class_test = round(len(testy[testy.values != predResTest.values]) / len(testy), 2)
    print("The best value of k and related misclassification rate for test data are: ")
    return [bestK,mis_class_test]



Run the generalised function on the music data set, using **RockOrNot** as the classifier, setting the seed at **123**, with **75**% of the data used for training and a set of values defined as k_vals.



In [24]:
k_vals = [1, 3, 5, 7, 9] 
kNN_classification(data,'RockOrNot',123,0.75,k_vals)   

The best value of k and related misclassification rate for test data are: 


[3, 0.06]